# **Εργασία 1 Κβαντική Μηχανική Μάθηση**
Ονοματεπώνυμο: Μανίκα Θεοδώρα

ΑΜ: 1115202100267

# Download libraries

In [ ]:
!pip install qiskit

In [ ]:
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.4 MB/s eta 0:00:00


In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer.primitives import SamplerV2
from qiskit.quantum_info import Statevector
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZFeatureMap
import numpy as np

# 1. Encoding on the basis of qubit states

In [ ]:
# Create the 3-qubit circuit
qc = QuantumCircuit(3)

# Step 1: Hadamard on q0 to create superposition
qc.h(0)

# Step 2: CNOTs to entangle q1 and q2 with q0
qc.cx(0, 1)  # q0 controls q1
qc.cx(0, 2)  # q0 controls q2

# Step 3: Second Hadamard on q0 to split the terms
qc.h(0)

print(qc)

# Add measurements (optional but useful for verification)
qc.measure_all()

# Construct an ideal simulator with SamplerV2
sampler = SamplerV2()
job = sampler.run([qc], shots=128)  # Run 128 simulations

# Get the results
result_ideal = job.result()
counts_ideal = result_ideal[0].data.meas.get_counts()

print('Counts(ideal):', counts_ideal)

     ┌───┐          ┌───┐
q_0: ┤ H ├──■────■──┤ H ├
     └───┘┌─┴─┐  │  └───┘
q_1: ─────┤ X ├──┼───────
          └───┘┌─┴─┐     
q_2: ──────────┤ X ├─────
               └───┘     
Counts(ideal): {'001': 37, '000': 29, '110': 23, '111': 39}


# 2. Encoding on the amplitude of qubit states

In [ ]:
def encode_single_qubit(x, y):
    # Normalize the vector
    norm = np.sqrt(x**2 + y**2)
    x_norm, y_norm = x/norm, y/norm

    # Calculate rotation angle θ = 2*arccos(x)
    theta = 2 * np.arccos(x_norm)

    # Create circuit
    qc = QuantumCircuit(1)
    qc.ry(theta, 0)

    # Get statevector
    state = Statevector.from_instruction(qc)
    return qc, state

# Example usage
x, y = 0.6, 0.8
qc, state = encode_single_qubit(x, y)
print("Circuit:")
print(qc.draw())
print("\nStatevector:", state)

Circuit:
   ┌────────────┐
q: ┤ Ry(1.8546) ├
   └────────────┘

Statevector: Statevector([0.6+0.j, 0.8+0.j],
            dims=(2,))


In [ ]:
def encode_two_qubits(x, y, z, w):
    # Normalize the vector
    norm = np.sqrt(x**2 + y**2 + z**2 + w**2)
    x_norm, y_norm, z_norm, w_norm = x/norm, y/norm, z/norm, w/norm

    # Calculate angles
    theta1 = 2 * np.arccos(np.sqrt(x_norm**2 + y_norm**2))
    theta2 = 2 * np.arctan2(y_norm, x_norm)
    theta3 = 2 * np.arctan2(w_norm, z_norm)

    # Create circuit
    qc = QuantumCircuit(2)
    qc.ry(theta1, 1)
    qc.x(1)
    qc.cry(theta2, 1, 0)
    qc.x(1)
    qc.cry(theta3, 1, 0)


    # Get statevector
    state = Statevector.from_instruction(qc)
    return qc, state

# Test with given vector
x, y, z, w = 0.1, 0.2, 0.3, 0.927
qc, state = encode_two_qubits(x, y, z, w)

# Redefine the normalized variables in the global scope
norm = np.sqrt(x**2 + y**2 + z**2 + w**2)
x_norm, y_norm, z_norm, w_norm = x/norm, y/norm, z/norm, w/norm

print("Circuit:")
print(qc.draw())
print("\nStatevector:", state)

Circuit:
                        ┌────────────┐     ┌────────────┐
q_0: ───────────────────┤ Ry(2.2143) ├─────┤ Ry(2.5156) ├
     ┌────────────┐┌───┐└─────┬──────┘┌───┐└─────┬──────┘
q_1: ┤ Ry(2.6904) ├┤ X ├──────■───────┤ X ├──────■───────
     └────────────┘└───┘              └───┘              

Statevector: Statevector([0.10003357+0.j, 0.20006713+0.j, 0.3001007 +0.j,
             0.92731117+0.j],
            dims=(2, 2))


# 3. Encoding on the time-evolution of qubit states

In [ ]:
# Create parameterized circuit
num_qubits = 4
params = ParameterVector('θ', length=num_qubits)
qc = QuantumCircuit(num_qubits)

for i in range(2):
    # First RX rotation layer
    for qubit in range(num_qubits):
        qc.rx(params[qubit], qubit)

    # First entanglement layer (forward)
    for qubit in range(num_qubits - 1):
        qc.cx(qubit, qubit + 1)

# Bind parameters
input_vector = np.array([0.976, 2.303, 1.391, 2.028])
hw_embedding = qc.assign_parameters(input_vector)

print("HardwareEfficient EmbeddingRx:")
print(hw_embedding.draw(fold=-1))

HardwareEfficient EmbeddingRx:
     ┌───────────┐     ┌───────────┐                                    
q_0: ┤ Rx(0.976) ├──■──┤ Rx(0.976) ├───────────────────■────────────────
     ├───────────┤┌─┴─┐└───────────┘┌───────────┐    ┌─┴─┐              
q_1: ┤ Rx(2.303) ├┤ X ├──────■──────┤ Rx(2.303) ├────┤ X ├──────■───────
     ├───────────┤└───┘    ┌─┴─┐    └───────────┘┌───┴───┴───┐┌─┴─┐     
q_2: ┤ Rx(1.391) ├─────────┤ X ├──────────■──────┤ Rx(1.391) ├┤ X ├──■──
     ├───────────┤         └───┘        ┌─┴─┐    ├───────────┤└───┘┌─┴─┐
q_3: ┤ Rx(2.028) ├──────────────────────┤ X ├────┤ Rx(2.028) ├─────┤ X ├
     └───────────┘                      └───┘    └───────────┘     └───┘


In [ ]:
# Create parameterized circuit
num_qubits = 4
params = ParameterVector('θ', length=num_qubits)
qc = QuantumCircuit(num_qubits)

for i in range(4):
    qc.h(i)
    qc.p(2 * params[i], i)
    qc.h(i)
    qc.p(2 * params[i], i)

# Bind parameters
input_vector = np.array([0.976, 2.303, 1.391, 2.028])
zmap_embedding = qc.assign_parameters(input_vector)

print("ZFeatureMap:")
print(zmap_embedding.draw(fold=-1))

ZFeatureMap:
     ┌───┐┌──────────┐┌───┐┌──────────┐
q_0: ┤ H ├┤ P(1.952) ├┤ H ├┤ P(1.952) ├
     ├───┤├──────────┤├───┤├──────────┤
q_1: ┤ H ├┤ P(4.606) ├┤ H ├┤ P(4.606) ├
     ├───┤├──────────┤├───┤├──────────┤
q_2: ┤ H ├┤ P(2.782) ├┤ H ├┤ P(2.782) ├
     ├───┤├──────────┤├───┤├──────────┤
q_3: ┤ H ├┤ P(4.056) ├┤ H ├┤ P(4.056) ├
     └───┘└──────────┘└───┘└──────────┘


In [ ]:
# Create ZFeatureMap with 4 features
z_map = ZFeatureMap(feature_dimension=4, reps=1)

# Bind parameters
z_embedding = z_map.assign_parameters(input_vector)

print("\nZFeatureMap from qiskit:")
print(z_embedding.draw())


ZFeatureMap Circuit:
     ┌───────────────────────────────────────┐
q_0: ┤0                                      ├
     │                                       │
q_1: ┤1                                      ├
     │  ZFeatureMap(0.976,2.303,1.391,2.028) │
q_2: ┤2                                      ├
     │                                       │
q_3: ┤3                                      ├
     └───────────────────────────────────────┘


In [ ]:
def print_state(qc):
    state = Statevector.from_instruction(qc)
    print("Final state (first 4 terms):")
    print(np.round(state.data[:4], 3))

print("\nHardwareEfficient State:")
print_state(hw_embedding)

print("\nZFeatureMap State:")
print_state(zmap_embedding)

print("\nZFeatureMap State:")
print_state(z_embedding)


HardwareEfficient State:
Final state (first 4 terms):
[ 0.206-0.151j -0.004-0.15j  -0.178+0.135j  0.238+0.141j]

ZFeatureMap State:
Final state (first 4 terms):
[0.027+0.012j 0.031+0.031j 0.031+0.01j  0.038+0.031j]

ZFeatureMap State:
Final state (first 4 terms):
[ 0.25 +0.j    -0.093+0.232j -0.027-0.249j  0.241+0.068j]
